In [1]:
from eve import Eve
from eve_plus import EvePlus
from models import *


import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from utils import lr_down_linearly, lr_down_cyclically_e4, lr_down_fixed_step, lr_down_sqrt, lr_down_linearly_v3
from utils import lr_up_linearly, lr_down_cyclically_e8, lr_down_cyclically_a4,lr_down_cyclically_a8


%reload_ext autoreload
%autoreload 2


In [ ]:
### pytorch cifar

In [2]:
# variables
batch_size = 128
cuda = torch.cuda.is_available()


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# load data
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)





Files already downloaded and verified
Files already downloaded and verified


In [36]:
len(train_loader.dataset)

50000

In [3]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_d_t = []
    total_lost_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        # change eve_plus and utils
        if epoch < 27:
            optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
        else:
            lr_initial = 0.01
            optimizer = lr_down_cyclically_e4(optimizer, lr_initial, epoch, batch_idx)
#         optimizer = lr_down_sqrt(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, d_t, output = optimizer.step(closure)
        loss_value = loss.data[0]
        total_loss += loss_value / len(train_loader)
        total_lost_list.append(loss_value)
        total_d_t.append(d_t)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} Accuracy: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter,),
                end="")
    return total_loss, total_d_t, total_lost_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy


In [4]:
import time
print ('EVE')
eve_loss = []
eve_loss_list = []
eve_test_loss = []
eve_trn_acc = []
eve_test_acc = []
lr_lst = []
eve_dt = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = EvePlus(model.parameters(), lr=0.01)
epochs = 54
a = time.time()
for i in range(1, epochs + 1):
    train_loss, dt, loss_list, trn_acc, lr_per_epoch= train(i, model, optimizer)
    eve_loss.append(train_loss)
    eve_dt += dt
    eve_loss_list += loss_list
    lr_lst += lr_per_epoch
    eve_trn_acc.append(trn_acc)
    test_loss, test_accuracy = test(i, model)
    eve_test_loss.append(test_loss)
    eve_test_acc.append(test_accuracy)
b = time.time()
print (b-a)

EVE
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.56 Accuracy: 0.3708 lr: 0.00096329833349388397
Test set: Average loss: 1.7771, Accuracy: 4163/10000 (41.63%)
Train Epoch: 2 [35840/50000 (71.61%)] Loss: 0.805 Accuracy: 0.4293 lr: 0.00093703148425787121

Process Process-6:
KeyboardInterrupt
Process Process-5:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/queues.py", line 341, in get


KeyboardInterrupt: 

In [8]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_d_t = []
    total_lost_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        # change eve_plus and utils
        if epoch < 27:
            optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
        else:
            lr_initial = 0.005
            optimizer = lr_down_cyclically_e4(optimizer, lr_initial, epoch, batch_idx)
#         optimizer = lr_down_sqrt(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, d_t, output = optimizer.step(closure)
        loss_value = loss.data[0]
        total_loss += loss_value / len(train_loader)
        total_lost_list.append(loss_value)
        total_d_t.append(d_t)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} Accuracy: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter,),
                end="")
    return total_loss, total_d_t, total_lost_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy



In [9]:
import time
print ('EVE')
eve_loss005 = []
eve_loss_list005 = []
eve_test_loss005 = []
eve_trn_acc005 = []
eve_test_acc005 = []
lr_lst005 = []
eve_dt005 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = EvePlus(model.parameters(), lr=0.005)
epochs = 54
a = time.time()
for i in range(1, epochs + 1):
    train_loss, dt, loss_list, trn_acc, lr_per_epoch= train(i, model, optimizer)
    eve_loss005.append(train_loss)
    eve_dt005 += dt
    eve_loss_list005 += loss_list
    lr_lst005 += lr_per_epoch
    eve_trn_acc005.append(trn_acc)
    test_loss, test_accuracy = test(i, model)
    eve_test_loss005.append(test_loss)
    eve_test_acc005.append(test_accuracy)
b = time.time()
print (b-a)

EVE
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.54 Accuracy: 0.38528 lr: 0.0009632983334938837
Test set: Average loss: 1.4819, Accuracy: 4642/10000 (46.42%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.04 Accuracy: 0.60078 lr: 0.0009283327144448571
Test set: Average loss: 1.0430, Accuracy: 6450/10000 (64.50%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss: 0.845 Accuracy: 0.68172 lr: 0.0008958165367732688
Test set: Average loss: 0.9282, Accuracy: 6935/10000 (69.35%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.725 Accuracy: 0.72672 lr: 0.0008655011251514627
Test set: Average loss: 0.6635, Accuracy: 7827/10000 (78.27%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss: 0.612 Accuracy: 0.76936 lr: 0.0008371703641691083
Test set: Average loss: 0.6241, Accuracy: 7867/10000 (78.67%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 0.528 Accuracy: 0.79772 lr: 0.0008106355382619975
Test set: Average loss: 0.6313, Accuracy: 7882/10000 (78.82%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 0.479 Accuracy: 

Train Epoch: 53 [48640/50000 (97.19%)] Loss: 0.0292 Accuracy: 0.96518 lr: 0.00128516624040917146
Test set: Average loss: 0.3529, Accuracy: 9213/10000 (92.13%)
Train Epoch: 54 [48640/50000 (97.19%)] Loss: 0.0192 Accuracy: 0.96916 lr: 3.516624040916582e-052
Test set: Average loss: 0.3499, Accuracy: 9235/10000 (92.35%)
2929.310095310211


In [10]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_d_t = []
    total_lost_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        # change eve_plus and utils
        if epoch < 27:
            optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
        else:
            lr_initial = 0.01
            optimizer = lr_down_cyclically_e8(optimizer, lr_initial, epoch, batch_idx)
#         optimizer = lr_down_sqrt(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, d_t, output = optimizer.step(closure)
        loss_value = loss.data[0]
        total_loss += loss_value / len(train_loader)
        total_lost_list.append(loss_value)
        total_d_t.append(d_t)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} Accuracy: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter,),
                end="")
    return total_loss, total_d_t, total_lost_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy




In [11]:
import time
print ('EVE')
eve_loss01e8 = []
eve_loss_list01e8 = []
eve_test_loss01e8 = []
eve_trn_acc01e8 = []
eve_test_acc01e8 = []
lr_lst01e8 = []
eve_dt01e8 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = EvePlus(model.parameters(), lr=0.01)
epochs = 50
a = time.time()
for i in range(1, epochs + 1):
    train_loss, dt, loss_list, trn_acc, lr_per_epoch= train(i, model, optimizer)
    eve_loss01e8.append(train_loss)
    eve_dt01e8 += dt
    eve_loss_list01e8 += loss_list
    lr_lst01e8 += lr_per_epoch
    eve_trn_acc01e8.append(trn_acc)
    test_loss, test_accuracy = test(i, model)
    eve_test_loss01e8.append(test_loss)
    eve_test_acc01e8.append(test_accuracy)
b = time.time()
print (b-a)

EVE
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.65 Accuracy: 0.32368 lr: 0.0009632983334938837
Test set: Average loss: 1.7725, Accuracy: 3727/10000 (37.27%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.12 Accuracy: 0.5705 lr: 0.00092833271444485719
Test set: Average loss: 1.1634, Accuracy: 6015/10000 (60.15%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss: 0.881 Accuracy: 0.66316 lr: 0.0008958165367732688
Test set: Average loss: 1.3501, Accuracy: 5742/10000 (57.42%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.749 Accuracy: 0.71758 lr: 0.0008655011251514627
Test set: Average loss: 0.6971, Accuracy: 7615/10000 (76.15%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss: 0.633 Accuracy: 0.76036 lr: 0.0008371703641691083
Test set: Average loss: 0.6484, Accuracy: 7850/10000 (78.50%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 0.555 Accuracy: 0.78716 lr: 0.0008106355382619975
Test set: Average loss: 0.6327, Accuracy: 7915/10000 (79.15%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 0.494 Accuracy: 

In [12]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_d_t = []
    total_lost_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        # change eve_plus and utils
        if epoch < 27:
            optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
        else:
            lr_initial = 0.005
            optimizer = lr_down_cyclically_e8(optimizer, lr_initial, epoch, batch_idx)
#         optimizer = lr_down_sqrt(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, d_t, output = optimizer.step(closure)
        loss_value = loss.data[0]
        total_loss += loss_value / len(train_loader)
        total_lost_list.append(loss_value)
        total_d_t.append(d_t)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} Accuracy: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter,),
                end="")
    return total_loss, total_d_t, total_lost_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy





In [13]:
import time
print ('EVE')
eve_loss005e8 = []
eve_loss_list005e8 = []
eve_test_loss005e8 = []
eve_trn_acc005e8 = []
eve_test_acc005e8 = []
lr_lst005e8 = []
eve_dt005e8 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = EvePlus(model.parameters(), lr=0.005)
epochs = 50
a = time.time()
for i in range(1, epochs + 1):
    train_loss, dt, loss_list, trn_acc, lr_per_epoch= train(i, model, optimizer)
    eve_loss005e8.append(train_loss)
    eve_dt005e8 += dt
    eve_loss_list005e8 += loss_list
    lr_lst005e8 += lr_per_epoch
    eve_trn_acc005e8.append(trn_acc)
    test_loss, test_accuracy = test(i, model)
    eve_test_loss005e8.append(test_loss)
    eve_test_acc005e8.append(test_accuracy)
b = time.time()
print (b-a)

EVE
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.57 Accuracy: 0.36898 lr: 0.0009632983334938837
Test set: Average loss: 1.9931, Accuracy: 3704/10000 (37.04%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.06 Accuracy: 0.59182 lr: 0.0009283327144448571
Test set: Average loss: 1.0109, Accuracy: 6549/10000 (65.49%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss: 0.857 Accuracy: 0.67424 lr: 0.0008958165367732688
Test set: Average loss: 1.1911, Accuracy: 6237/10000 (62.37%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.732 Accuracy: 0.72316 lr: 0.0008655011251514627
Test set: Average loss: 0.6944, Accuracy: 7666/10000 (76.66%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss: 0.625 Accuracy: 0.76632 lr: 0.0008371703641691083
Test set: Average loss: 0.7819, Accuracy: 7426/10000 (74.26%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 0.538 Accuracy: 0.7951 lr: 0.00081063553826199752
Test set: Average loss: 0.5302, Accuracy: 8248/10000 (82.48%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 0.473 Accuracy: 

In [14]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_d_t = []
    total_lost_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
#         # change eve_plus and utils
#         if epoch < 27:
#             optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
#         else:
        lr_initial = 0.01
        optimizer = lr_down_cyclically_a8(optimizer, lr_initial, epoch, batch_idx)
#         optimizer = lr_down_sqrt(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, d_t, output = optimizer.step(closure)
        loss_value = loss.data[0]
        total_loss += loss_value / len(train_loader)
        total_lost_list.append(loss_value)
        total_d_t.append(d_t)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} Accuracy: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter,),
                end="")
    return total_loss, total_d_t, total_lost_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy






In [15]:
import time
print ('EVE')
eve_loss01a8 = []
eve_loss_list01a8 = []
eve_test_loss01a8 = []
eve_trn_acc01a8 = []
eve_test_acc01a8 = []
lr_lst01a8 = []
eve_dt01a8 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = EvePlus(model.parameters(), lr=0.01)
epochs = 56
a = time.time()
for i in range(1, epochs + 1):
    train_loss, dt, loss_list, trn_acc, lr_per_epoch= train(i, model, optimizer)
    eve_loss01a8.append(train_loss)
    eve_dt01a8 += dt
    eve_loss_list01a8 += loss_list
    lr_lst01a8 += lr_per_epoch
    eve_trn_acc01a8.append(trn_acc)
    test_loss, test_accuracy = test(i, model)
    eve_test_loss01a8.append(test_loss)
    eve_test_acc01a8.append(test_accuracy)
b = time.time()
print (b-a)

EVE
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  2.02 Accuracy: 0.18244 lr: 0.007570332480818221
Test set: Average loss: 1.8064, Accuracy: 2746/10000 (27.46%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.53 Accuracy: 0.37812 lr: 0.005070332480818327
Test set: Average loss: 1.4938, Accuracy: 4553/10000 (45.53%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss:  1.12 Accuracy: 0.56628 lr: 0.0025703324808183435
Test set: Average loss: 1.2340, Accuracy: 5830/10000 (58.30%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.862 Accuracy: 0.67128 lr: 7.033248081833165e-054
Test set: Average loss: 0.7559, Accuracy: 7395/10000 (73.95%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss: 6.97e+03 Accuracy: 0.13336 lr: -0.0024296675191816763
Test set: Average loss: 40628.6795, Accuracy: 1000/10000 (10.00%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 1.97e+05 Accuracy: 0.09768 lr: -0.0049296675191816734
Test set: Average loss: 482628.7366, Accuracy: 1000/10000 (10.00%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 

Train Epoch: 51 [48640/50000 (97.19%)] Loss: 5.8e+04 Accuracy: 0.12076 lr: 0.00257033248081834385
Test set: Average loss: 47534.8841, Accuracy: 1175/10000 (11.75%)
Train Epoch: 52 [48640/50000 (97.19%)] Loss: 5.33e+04 Accuracy: 0.12208 lr: 7.033248081833165e-054
Test set: Average loss: 80700.3841, Accuracy: 1338/10000 (13.38%)
Train Epoch: 53 [48640/50000 (97.19%)] Loss: 5.48e+04 Accuracy: 0.11946 lr: -0.0024296675191816763
Test set: Average loss: 97493.7909, Accuracy: 1451/10000 (14.51%)
Train Epoch: 54 [48640/50000 (97.19%)] Loss: 2.09e+05 Accuracy: 0.13806 lr: -0.0049296675191816765
Test set: Average loss: 1717314.8194, Accuracy: 1402/10000 (14.02%)
Train Epoch: 55 [48640/50000 (97.19%)] Loss: 3.58e+06 Accuracy: 0.08214 lr: -0.0074296675191815644
Test set: Average loss: 4485906.5000, Accuracy: 525/10000 (5.25%)
Train Epoch: 56 [48640/50000 (97.19%)] Loss: 4.56e+06 Accuracy: 0.05972 lr: -0.009929667519181746
Test set: Average loss: 4661909.6650, Accuracy: 553/10000 (5.53%)
2995.01111

In [95]:
len(eve_test_acc)

52

vgg19 1epoch 107s
google net 1epoch 106s
105.70495510101318
2748.2364859580994


### adam rms ada

In [16]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_loss_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer = lr_down_linearly_v3(optimizer, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, output = optimizer.step(closure)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        loss_value = loss.data[0]
        total_loss +=  loss_value/ len(train_loader)
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        
        total_loss_list.append(loss_value)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} acc: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                       batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter),
                end="")
    return total_loss, total_loss_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy

In [17]:
print("Adam")
adam_loss = []
adam_loss_list = []
adam_test_loss = []
adam_test_acc = []
adam_trn_acc = []
lr_lst = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
epochs = 50
for i in range(1, epochs + 1):
    total_loss, total_loss_list, trn_acc, lr_per_epoch = train(i, model, optimizer)
    adam_loss.append(total_loss)
    adam_trn_acc.append(trn_acc)
    adam_loss_list += total_loss_list
    lr_lst += lr_per_epoch
    test_loss, test_acc = test(i, model)
    adam_test_loss.append(test_loss)
    adam_test_acc.append(test_acc)


# plot(eve_loss, adam_loss, "eve_loss.png", "training loss")
# plot(eve_test_loss, adam_test_loss, "eve_test_loss.png", "testing loss")

Adam
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.99 acc: 0.2005 lr: 0.00096329833349388327
Test set: Average loss: 1.7718, Accuracy: 3091/10000 (30.91%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.47 acc: 0.40196 lr: 0.0009283327144448571
Test set: Average loss: 1.4811, Accuracy: 4477/10000 (44.77%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss:  1.14 acc: 0.55326 lr: 0.0008958165367732688
Test set: Average loss: 1.5516, Accuracy: 5087/10000 (50.87%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.904 acc: 0.6515 lr: 0.00086550112515146273
Test set: Average loss: 1.3668, Accuracy: 5516/10000 (55.16%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss:  0.77 acc: 0.70022 lr: 0.0008371703641691083
Test set: Average loss: 1.1265, Accuracy: 6220/10000 (62.20%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss:  0.67 acc: 0.73896 lr: 0.0008106355382619975
Test set: Average loss: 0.9231, Accuracy: 6862/10000 (68.62%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 0.602 acc: 0.7645 lr: 0.00078573112280977452


In [35]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_loss_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        lr_initial = 0.005
        optimizer = lr_down_cyclically_a8(optimizer, lr_initial, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, output = optimizer.step(closure)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        loss_value = loss.data[0]
        total_loss +=  loss_value/ len(train_loader)
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        
        total_loss_list.append(loss_value)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} acc: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                       batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter),
                end="")
    return total_loss, total_loss_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy

In [36]:
print("Adam")
adam_loss005a8 = []
adam_loss_list005a8 = []
adam_test_loss005a8 = []
adam_test_acc005a8 = []
adam_trn_acc005a8 = []
lr_lst005a8 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = optim.SGD(model.parameters(), lr=0.01)
epochs = 56
for i in range(1, epochs + 1):
    total_loss, total_loss_list, trn_acc, lr_per_epoch = train(i, model, optimizer)
    adam_loss005a8.append(total_loss)
    adam_trn_acc005a8.append(trn_acc)
    adam_loss_list005a8 += total_loss_list
    lr_lst005a8 += lr_per_epoch
    test_loss, test_acc = test(i, model)
    adam_test_loss005a8.append(test_loss)
    adam_test_acc005a8.append(test_acc)


# plot(eve_loss, adam_loss, "eve_loss.png", "training loss")
# plot(eve_test_loss, adam_test_loss, "eve_test_loss.png", "testing loss")

Adam
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.48 acc: 0.43556 lr: 0.0037851662404091106
Test set: Average loss: 1.3676, Accuracy: 5071/10000 (50.71%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:  1.11 acc: 0.57594 lr: 0.0025351662404091637
Test set: Average loss: 1.0373, Accuracy: 6277/10000 (62.77%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss: 0.949 acc: 0.63646 lr: 0.0012851662404091714
Test set: Average loss: 0.8854, Accuracy: 6845/10000 (68.45%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.859 acc: 0.6706 lr: 3.516624040916582e-0552
Test set: Average loss: 0.8271, Accuracy: 7044/10000 (70.44%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss:  19.8 acc: 0.19008 lr: -0.00121483375959083827
Test set: Average loss: 76.1031, Accuracy: 376/10000 (3.76%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 2.86e+02 acc: 0.0463 lr: -0.00246483375959083567
Test set: Average loss: 852.5474, Accuracy: 613/10000 (6.13%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 1.05e+04 acc: 0.07394 lr: -0.0037148337

Train Epoch: 51 [48640/50000 (97.19%)] Loss: 1.21e+18 acc: 0.0975 lr: 0.00128516624040917147
Test set: Average loss: 109622886057660912.0000, Accuracy: 1000/10000 (10.00%)
Train Epoch: 52 [48640/50000 (97.19%)] Loss: 4.64e+15 acc: 0.09782 lr: 3.516624040916582e-052
Test set: Average loss: 4.4796, Accuracy: 1000/10000 (10.00%)
Train Epoch: 53 [48640/50000 (97.19%)] Loss:  4.42 acc: 0.09744 lr: -0.00121483375959083827
Test set: Average loss: 2096012872667627.5000, Accuracy: 1000/10000 (10.00%)
Train Epoch: 54 [48640/50000 (97.19%)] Loss: 9.9e+17 acc: 0.09726 lr: -0.00246483375959083567
Test set: Average loss: 2795414250400723968.0000, Accuracy: 1000/10000 (10.00%)
Train Epoch: 55 [48640/50000 (97.19%)] Loss: 1.01e+19 acc: 0.09754 lr: -0.0037148337595907826
Test set: Average loss: 30182594628081311744.0000, Accuracy: 1000/10000 (10.00%)
Train Epoch: 56 [48640/50000 (97.19%)] Loss: 2e+20 acc: 0.09758 lr: -0.0049648337595908730365
Test set: Average loss: 795058309752018042880.0000, Accuracy

In [ ]:
def train(epoch, model, optimizer):
    model.train()
    total_loss = 0
    total_loss_list = []
    train_correct = 0
    lr_per_epoch = []
    for batch_idx, (data, target) in enumerate(train_loader):
        lr_initial = 0.005
        optimizer = lr_down_cyclically_a4(optimizer, lr_initial, epoch, batch_idx)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        def closure():
            optimizer.zero_grad()  # reset reset optimizer
            output = model(data)
            loss = F.cross_entropy(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss, output

        loss, output = optimizer.step(closure)
        pred = output.data.max(1)[1]
        train_correct += pred.eq(target.data).cpu().sum()
        dataset_num = len(train_loader.dataset)
        trn_acc = train_correct / dataset_num
        loss_value = loss.data[0]
        total_loss +=  loss_value/ len(train_loader)
        lr_per_iter = optimizer.param_groups[0]['lr']
        lr_per_epoch.append(lr_per_iter)
        
        total_loss_list.append(loss_value)
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:>4.2%})] Loss: {:>5.3} acc: {} lr: {}'.format(
                epoch, batch_idx * len(data), dataset_num,
                       batch_idx / len(train_loader), total_loss, trn_acc, lr_per_iter),
                end="")
    return total_loss, total_loss_list, trn_acc, lr_per_epoch


def test(epoch, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)  # loss function already averages over batch size
    test_accuracy = correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2%})'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))
    return test_loss, test_accuracy

In [ ]:
print("Adam")
adam_loss005a4 = []
adam_loss_list005a4 = []
adam_test_loss005a4 = []
adam_test_acc005a4 = []
adam_trn_acc005a4 = []
lr_lst005a4 = []
torch.manual_seed(233)
model = VGG('VGG16')
if cuda:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
optimizer = optim.SGD(model.parameters(), lr=0.05)
epochs = 52
for i in range(1, epochs + 1):
    total_loss, total_loss_list, trn_acc, lr_per_epoch = train(i, model, optimizer)
    adam_loss005a4.append(total_loss)
    adam_trn_acc005a4.append(trn_acc)
    adam_loss_list005a4 += total_loss_list
    lr_lst005a4 += lr_per_epoch
    test_loss, test_acc = test(i, model)
    adam_test_loss005a4.append(test_loss)
    adam_test_acc005a4.append(test_acc)


# plot(eve_loss, adam_loss, "eve_loss.png", "training loss")
# plot(eve_test_loss, adam_test_loss, "eve_test_loss.png", "testing loss")

Adam
Train Epoch: 1 [48640/50000 (97.19%)] Loss:  1.48 acc: 0.43238 lr: 0.0037851662404091106
Test set: Average loss: 1.2612, Accuracy: 5477/10000 (54.77%)
Train Epoch: 2 [48640/50000 (97.19%)] Loss:   1.1 acc: 0.5782 lr: 0.00253516624040916376
Test set: Average loss: 0.9998, Accuracy: 6383/10000 (63.83%)
Train Epoch: 3 [48640/50000 (97.19%)] Loss: 0.943 acc: 0.63976 lr: 0.0012851662404091714
Test set: Average loss: 0.8878, Accuracy: 6828/10000 (68.28%)
Train Epoch: 4 [48640/50000 (97.19%)] Loss: 0.854 acc: 0.67366 lr: 3.516624040916582e-052
Test set: Average loss: 0.8264, Accuracy: 7084/10000 (70.84%)
Train Epoch: 5 [48640/50000 (97.19%)] Loss: 0.893 acc: 0.65904 lr: 0.0037851662404091106
Test set: Average loss: 0.8482, Accuracy: 7004/10000 (70.04%)
Train Epoch: 6 [48640/50000 (97.19%)] Loss: 0.768 acc: 0.70152 lr: 0.0025351662404091637
Test set: Average loss: 0.7604, Accuracy: 7298/10000 (72.98%)
Train Epoch: 7 [48640/50000 (97.19%)] Loss: 0.686 acc: 0.73524 lr: 0.0012851662404091714

In [40]:
import pickle

with open("cycle-loss-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(adam_loss005a8, fp)
with open("cycle-loss-list-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(adam_loss_list005a8, fp)
with open("cycle-test-loss-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(adam_test_loss005a8, fp)
with open("cycle-test-acc-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(adam_test_acc005a8, fp)
with open("cycle-trn-acc-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(adam_trn_acc005a8, fp)
with open("cycle-lr-lst-model-vgg16-0.005-8-56.txt", 'wb') as fp:
    pickle.dump(lr_lst005a8, fp)

## manual_seed weigtht initialization test 

In [8]:
# print("Eve")
# eve_loss = []
# eve_test_loss = []

torch.manual_seed(23)


model_1 = Net()
model_1.cuda()
model_1.conv1.weight

# if cuda:
#     model.cuda()
# optimizer = Eve(model.parameters())
# for i in range(1, epochs + 1):
#     eve_loss.append(train(i, model, optimizer))
#     eve_test_loss.append(test(i, model))

# print("Adam")
# adam_loss = []
# adam_test_loss = []
# model = Net()
# if cuda:
#     model.cuda()
# optimizer = optim.Adam(model.parameters())
# for i in range(1, epochs + 1):
#     adam_loss.append(train(i, model, optimizer))
#     adam_test_loss.append(test(i, model))

# plot(eve_loss)
# plot(eve_test_loss, adam_test_loss, "eve_test_loss.png", "testing loss")

Parameter containing:
(0 ,0 ,.,.) = 
  0.0067  0.0651  0.1720
 -0.1212  0.1022  0.1084
 -0.0838  0.0284 -0.1074

(0 ,1 ,.,.) = 
 -0.0224  0.0717 -0.1216
 -0.1281 -0.1540 -0.0414
 -0.1079  0.0454  0.1283

(0 ,2 ,.,.) = 
 -0.0339 -0.0282 -0.1915
  0.0980  0.1478  0.1912
  0.1482 -0.1924 -0.0768

(1 ,0 ,.,.) = 
  0.1832  0.0345 -0.0311
  0.1841  0.0699  0.1328
  0.0972 -0.1674  0.0304

(1 ,1 ,.,.) = 
 -0.0790 -0.0404 -0.0816
 -0.0079  0.1241 -0.0420
  0.0486  0.0643 -0.1499

(1 ,2 ,.,.) = 
  0.0986 -0.1922  0.1284
  0.1702 -0.1262 -0.1380
 -0.1108 -0.0302  0.1442

(2 ,0 ,.,.) = 
 -0.0591 -0.0259  0.1423
  0.1448 -0.0275 -0.0113
  0.1265 -0.0760  0.0839

(2 ,1 ,.,.) = 
  0.1029 -0.1466 -0.1885
  0.0371  0.0678 -0.1425
 -0.0109 -0.1626  0.0516

(2 ,2 ,.,.) = 
  0.1275  0.1618 -0.0137
 -0.1418 -0.1301  0.1306
  0.0185  0.0789  0.0341

(3 ,0 ,.,.) = 
  0.1801  0.1053  0.1556
  0.0600 -0.0797  0.0222
 -0.1357 -0.1238 -0.1381

(3 ,1 ,.,.) = 
 -0.0998  0.1749  0.0023
 -0.1923 -0.0397 -0.0806
 -0

In [15]:
model_2 = Net()
model_2.cuda()
model_2.conv1.weight

Parameter containing:
(0 ,0 ,.,.) = 
  0.0067  0.0651  0.1720
 -0.1212  0.1022  0.1084
 -0.0838  0.0284 -0.1074

(0 ,1 ,.,.) = 
 -0.0224  0.0717 -0.1216
 -0.1281 -0.1540 -0.0414
 -0.1079  0.0454  0.1283

(0 ,2 ,.,.) = 
 -0.0339 -0.0282 -0.1915
  0.0980  0.1478  0.1912
  0.1482 -0.1924 -0.0768

(1 ,0 ,.,.) = 
  0.1832  0.0345 -0.0311
  0.1841  0.0699  0.1328
  0.0972 -0.1674  0.0304

(1 ,1 ,.,.) = 
 -0.0790 -0.0404 -0.0816
 -0.0079  0.1241 -0.0420
  0.0486  0.0643 -0.1499

(1 ,2 ,.,.) = 
  0.0986 -0.1922  0.1284
  0.1702 -0.1262 -0.1380
 -0.1108 -0.0302  0.1442

(2 ,0 ,.,.) = 
 -0.0591 -0.0259  0.1423
  0.1448 -0.0275 -0.0113
  0.1265 -0.0760  0.0839

(2 ,1 ,.,.) = 
  0.1029 -0.1466 -0.1885
  0.0371  0.0678 -0.1425
 -0.0109 -0.1626  0.0516

(2 ,2 ,.,.) = 
  0.1275  0.1618 -0.0137
 -0.1418 -0.1301  0.1306
  0.0185  0.0789  0.0341

(3 ,0 ,.,.) = 
  0.1801  0.1053  0.1556
  0.0600 -0.0797  0.0222
 -0.1357 -0.1238 -0.1381

(3 ,1 ,.,.) = 
 -0.0998  0.1749  0.0023
 -0.1923 -0.0397 -0.0806
 -0

## write 写

In [30]:
type(eve_loss)

list

In [24]:
import pickle

with open("eve-loss-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(eve_loss, fp)
with open("eve-loss-list-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(eve_loss_list, fp)
with open("eve-test-loss-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(eve_test_loss, fp)
with open("eve-test-acc-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(eve_test_acc, fp)
with open("eve-trn-acc-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(eve_trn_acc, fp)
# with open("eve-dt-model-vgg16-0.001-0.001-v3.txt", 'wb') as fp:
#     pickle.dump(eve_dt, fp)
    
with open("eve-lr-lst-model-vgg16-0.001.txt", 'wb') as fp:
    pickle.dump(lr_lst, fp)

In [17]:
import pickle

with open("cycle-loss-model-vgg16-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_loss, fp)
with open("cycle-loss-list-model-vgg16-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_loss_list, fp)
with open("cycle-test_loss-model-vgg16-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_test_loss, fp)
with open("cycle-test-acc-vgg16-0.0007-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_test_acc, fp)
with open("cycle-dt-model-vgg16-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_dt, fp)
with open("cycle-trn-acc-model-vgg16-0-0.01-4-52-with-dt.txt", 'wb') as fp:
    pickle.dump(eve_trn_acc, fp)